#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Get Cloud Project ID
To run this recipe [requires a Google Cloud Project](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md), this only needs to be done once, then click play.


In [ ]:
CLOUD_PROJECT = 'PASTE PROJECT ID HERE'

print("Cloud Project Set To: %s" % CLOUD_PROJECT)


#3. Get Client Credentials
To read and write to various endpoints requires [downloading client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md), this only needs to be done once, then click play.


In [ ]:
CLIENT_CREDENTIALS = 'PASTE CREDENTIALS HERE'

print("Client Credentials Set To: %s" % CLIENT_CREDENTIALS)


#4. Enter Monthly Budget Mover Parameters
Apply the previous month's budget/spend delta to the current month.  Aggregate up the budget and spend from the previous month of each category declared then apply the delta of the spend and budget equally to each Line Item under that Category.
 1. No changes made can be made in DV360 from the start to the end of this process
 1. Make sure there is budget information for the current and previous month's IOs in DV360
 1. Make sure the provided spend report has spend data for every IO in the previous month
 1. Spend report must contain 'Revenue (Adv Currency)' and 'Insertion Order ID'
 1. There are no duplicate IO Ids in the categories outlined below
 1. This process must be ran during the month of the budget it is updating
 1. If you receive a 502 error then you must separate your jobs into two, because there is too much information being pulled in the sdf
 1. Manually run this job
 1. Once the job has completed go to the table for the new sdf and export to a csv
 1. Take the new sdf and upload it into DV360
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'recipe_name': '',  # 
  'recipe_timezone': 'America/Los_Angeles',  # Timezone for report dates.
  'auth_read': 'user',  # Credentials used for reading data.
  'auth_write': 'service',  # Credentials used for writing data.
  'partner_id': '',  # The sdf file types.
  'budget_categories': '{}',  # A dictionary to show which IO Ids go under which Category. {"CATEGORY1":[12345,12345,12345], "CATEGORY2":[12345,12345]}
  'excluded_ios': '',  # A comma separated list of Inserion Order Ids that should be exluded from the budget calculations
  'filter_ids': [],  # Comma separated list of filter ids for the request.
  'version': '5',  # The sdf version to be returned.
  'dataset': '',  # Dataset to be written to in BigQuery.
  'is_colab': True,  # Are you running this in Colab? (This will store the files in Colab instead of Bigquery)
}

print("Parameters Set To: %s" % FIELDS)


#5. Execute Monthly Budget Mover
This does NOT need to be modified unles you are changing the recipe, click play.


In [ ]:
from starthinker.util.project import project
from starthinker.script.parse import json_set_fields

USER_CREDENTIALS = '/content/user.json'

TASKS = [
  {
    'dataset': {
      'auth': 'user',
      'description': 'Create a dataset where data will be combined and transfored for upload.',
      'dataset': {'field': {'description': 'Place where tables will be created in BigQuery.','name': 'dataset','order': 1,'kind': 'string'}}
    }
  },
  {
    'dbm': {
      'auth': 'user',
      'report': {
        'filters': {
          'FILTER_ADVERTISER': {
            'values': {'field': {'description': 'The comma separated list of Advertiser Ids.','name': 'filter_ids','default': '','order': 7,'kind': 'integer_list'}}
          }
        },
        'body': {
          'params': {
            'type': 'TYPE_GENERAL',
            'groupBys': [
              'FILTER_ADVERTISER_CURRENCY',
              'FILTER_INSERTION_ORDER'
            ],
            'metrics': [
              'METRIC_REVENUE_ADVERTISER'
            ]
          },
          'timezoneCode': {'field': {'description': 'Timezone for report dates.','name': 'recipe_timezone','default': 'America/Los_Angeles','kind': 'timezone'}},
          'metadata': {
            'title': {'field': {'description': 'Name of report in DV360, should be unique.','name': 'recipe_name','order': 1,'prefix': 'Monthly_Budget_Mover_','kind': 'string'}},
            'dataRange': 'PREVIOUS_MONTH',
            'format': 'CSV'
          }
        },
        'timeout': 90
      },
      'delete': False
    }
  },
  {
    'monthly_budget_mover': {
      'budget_categories': {'field': {'description': 'A dictionary to show which IO Ids go under which Category. {"CATEGORY1":[12345,12345,12345], "CATEGORY2":[12345,12345]}','name': 'budget_categories','default': '{}','order': 3,'kind': 'json'}},
      'out_changes': {
        'file': '/content/log.csv',
        'bigquery': {
          'table': {'field': {'description': '','name': 'recipe_name','prefix': 'SDF_BUDGET_MOVER_LOG_','kind': 'string'}},
          'dataset': {'field': {'description': 'Dataset that you would like your output tables to be produced in.','name': 'dataset','default': '','order': 8,'kind': 'string'}},
          'disposition': 'WRITE_TRUNCATE',
          'schema': [
          ],
          'skip_rows': 0
        }
      },
      'out_new_sdf': {
        'file': '/content/new_sdf.csv',
        'bigquery': {
          'table': {'field': {'description': '','name': 'recipe_name','prefix': 'SDF_NEW_','kind': 'string'}},
          'dataset': {'field': {'description': 'Dataset that you would like your output tables to be produced in.','name': 'dataset','default': '','order': 8,'kind': 'string'}},
          'disposition': 'WRITE_TRUNCATE',
          'schema': [
          ],
          'skip_rows': 0
        }
      },
      'excluded_ios': {'field': {'description': 'A comma separated list of Inserion Order Ids that should be exluded from the budget calculations','name': 'excluded_ios','order': 4,'kind': 'integer_list'}},
      'auth': 'user',
      'report_name': {'field': {'description': 'Name of report in DV360, should be unique.','name': 'recipe_name','order': 1,'prefix': 'Monthly_Budget_Mover_','kind': 'string'}},
      'is_colab': {'field': {'description': 'Are you running this in Colab? (This will store the files in Colab instead of Bigquery)','name': 'is_colab','default': True,'order': 7,'kind': 'boolean'}},
      'out_old_sdf': {
        'file': '/content/old_sdf.csv',
        'bigquery': {
          'table': {'field': {'description': '','name': 'recipe_name','prefix': 'SDF_OLD_','kind': 'string'}},
          'dataset': {'field': {'description': 'Dataset that you would like your output tables to be produced in.','name': 'dataset','default': '','order': 8,'kind': 'string'}},
          'disposition': 'WRITE_TRUNCATE',
          'schema': [
          ],
          'skip_rows': 0
        }
      },
      'sdf': {
        'time_partitioned_table': False,
        'filter_type': 'FILTER_TYPE_ADVERTISER_ID',
        'read': {
          'filter_ids': {
            'values': {'field': {'description': 'Comma separated list of filter ids for the request.','name': 'filter_ids','default': [],'order': 4,'kind': 'integer_list'}},
            'single_cell': True
          }
        },
        'partner_id': {'field': {'description': 'The sdf file types.','name': 'partner_id','order': 1,'kind': 'integer'}},
        'auth': 'user',
        'table_suffix': '',
        'dataset': {'field': {'description': 'Dataset to be written to in BigQuery.','name': 'dataset','default': '','order': 6,'kind': 'string'}},
        'version': {'field': {'choices': ['SDF_VERSION_5','SDF_VERSION_5_1'],'description': 'The sdf version to be returned.','order': 6,'name': 'version','default': '5','kind': 'choice'}},
        'file_types': 'INSERTION_ORDER',
        'create_single_day_table': False
      }
    }
  }
]

json_set_fields(TASKS, FIELDS)

project.initialize(_recipe={ 'tasks':TASKS }, _project=CLOUD_PROJECT, _user=USER_CREDENTIALS, _client=CLIENT_CREDENTIALS, _verbose=True, _force=True)
project.execute(_force=True)
